## 1. 학습데이터의 월 판매량 및 모든 시차 피쳐, 예측 결과 (0,20) 범위로 clip

## 2. hyper parameter 지정
- n_estimators=50 # 랜덤 포레스트 속 결정 나무 개수
- criterion='squared_error' # 평가지표(default:mse)
- max_depth=9 # 결정 나무 최대 깊이 (default:None)
- min_samples_split=2 # 노드를 분할하기 위한 최소한의 샘플 수 (default:2)
- min_samples_leaf=1 # 리프 노드가 되기 위한 최소한의 샘플 수 (default:1)
- min_weight_fraction_leaf=0.0
- max_features='auto' # 무작위 선택 최대 feature 개수 (default:auto)
- max_leaf_nodes=None # 리프 노드의 최대 개수 (default:None)
- min_impurity_decrease=0.0
- bootstrap=True # 전체 feature 복원 추출
- oob_score=False
- n_jobs=-1 # 머신이 사용할 코어의 수..모든 코어 사용 (default:None)
- random_state=42 # 랜덤 시드값 고정
- verbose=0
- warm_start=False
- ccp_alpha=0.0
- max_samples=None

## 3. 사용 데이터
- data_1: 가장 기본 전처리

## 4. 제출 여부: Y
- score: 0.95336

In [ ]:
# 라이브러리 호출
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import folium
import time
from itertools import product 
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:
import os
os.chdir('/Users/sujiyang/Desktop/aiservice/kaggle_future_sales')

In [4]:
# 데이터 불러오기
data = pd.read_pickle('/Users/sujiyang/Desktop/aiservice/kaggle_future_sales/data_1.pkl')
data.head()

,month_id,shop_id,item_id,cnt_month,city_id,cat_id,main_cat,sub_cat,year,month,...,sub_cat_mean_cnt_3,price_mean_mean_cnt_1,price_mean_mean_cnt_2,price_mean_mean_cnt_3,item_id/city_id_mean_cnt_1,item_id/city_id_mean_cnt_2,item_id/city_id_mean_cnt_3,shop_id/cat_id_mean_cnt_1,shop_id/cat_id_mean_cnt_2,shop_id/cat_id_mean_cnt_3
827691,3,25,8092,3.0,13,40,11,6,2013,4,...,0.0,1.400024,1.365631,0.0,1.000000,2.142857,0.0,1.087262,1.048394,0.0
827692,3,25,7850,3.0,13,30,8,54,2013,4,...,0.0,1.479122,1.464680,0.0,3.714286,2.285714,0.0,3.651685,2.276650,0.0
827693,3,25,8051,3.0,13,66,14,47,2013,4,...,0.0,0.000000,0.000000,0.0,0.000000,0.428571,0.0,1.827586,1.389474,0.0
827694,3,25,8088,1.0,13,55,13,4,2013,4,...,0.0,1.479122,0.000000,0.0,0.285714,0.000000,0.0,0.598291,0.562107,0.0
827695,3,25,8089,1.0,13,55,13,4,2013,4,...,0.0,0.000000,0.000000,0.0,0.571429,0.000000,0.0,0.598291,0.562107,0.0


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8032887 entries, 827691 to 8860577
Data columns (total 29 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   month_id                    int8   
 1   shop_id                     int8   
 2   item_id                     int16  
 3   cnt_month                   float32
 4   city_id                     int8   
 5   cat_id                      int8   
 6   main_cat                    int8   
 7   sub_cat                     int8   
 8   year                        int16  
 9   month                       int8   
 10  sales_month                 int8   
 11  cnt_month_1                 float32
 12  cnt_month_2                 float32
 13  cnt_month_3                 float32
 14  item_id_mean_cnt_1          float32
 15  item_id_mean_cnt_2          float32
 16  item_id_mean_cnt_3          float32
 17  sub_cat_mean_cnt_1          float32
 18  sub_cat_mean_cnt_2          float32
 19  sub_cat_mean_cnt

In [5]:
# whole data clip(0~20)
data[['cnt_month', 'cnt_month_1','cnt_month_2', 'cnt_month_3', 'item_id_mean_cnt_1','item_id_mean_cnt_2', 'item_id_mean_cnt_3', 
        'sub_cat_mean_cnt_1','sub_cat_mean_cnt_2', 'sub_cat_mean_cnt_3', 'price_mean_mean_cnt_1',
       'price_mean_mean_cnt_2', 'price_mean_mean_cnt_3','item_id/city_id_mean_cnt_1', 'item_id/city_id_mean_cnt_2',
       'item_id/city_id_mean_cnt_3', 'shop_id/cat_id_mean_cnt_1','shop_id/cat_id_mean_cnt_2', 'shop_id/cat_id_mean_cnt_3']] = data[['cnt_month', 'cnt_month_1','cnt_month_2', 'cnt_month_3', 'item_id_mean_cnt_1','item_id_mean_cnt_2', 'item_id_mean_cnt_3', 
        'sub_cat_mean_cnt_1','sub_cat_mean_cnt_2', 'sub_cat_mean_cnt_3', 'price_mean_mean_cnt_1',
       'price_mean_mean_cnt_2', 'price_mean_mean_cnt_3','item_id/city_id_mean_cnt_1', 'item_id/city_id_mean_cnt_2',
       'item_id/city_id_mean_cnt_3', 'shop_id/cat_id_mean_cnt_1','shop_id/cat_id_mean_cnt_2', 'shop_id/cat_id_mean_cnt_3']].clip(0, 20)

In [6]:
# train/validation/test data split
X_train = data[data['month_id'] < 33].drop(['cnt_month'], axis=1)
y_train = data[data['month_id'] < 33]['cnt_month']

X_valid = data[data['month_id'] == 33].drop(['cnt_month'], axis=1)
y_valid = data[data['month_id'] == 33]['cnt_month']

X_test = data[data['month_id'] == 34].drop(['cnt_month'], axis=1)

In [7]:
rf_features = [c for c in data.columns if c not in 'cnt_month']

# 데이터 셋 지정
rf_train = X_train[rf_features]
rf_valid = X_valid[rf_features]
rf_test = X_test[rf_features]
 
# 모델 훈련
start = time.time()
rf_model = RandomForestRegressor(n_estimators=50, max_depth=9,random_state=42, n_jobs=-1)
rf_model.fit(rf_train, y_train)
end = time.time()
print(f'소요시간(s) : {end-start}')

소요시간(s) : 2537.5692749023438


In [8]:
# 예측 수행
rf_train_pred = rf_model.predict(rf_train).clip(0,20)
rf_valid_pred = rf_model.predict(rf_valid).clip(0,20)
rf_test_pred = rf_model.predict(rf_test).clip(0, 20) # 실제 타겟값이 클립되어있으므로 동일하게 클립

In [9]:
# 제출 파일 생성
df_test = pd.read_csv('./test.csv')
submission = pd.DataFrame({"ID": df_test.index, "item_cnt_month": rf_test_pred})
submission.to_csv('RF_submission_1.csv', index=False)

In [15]:
# 피쳐 중요도를 가지는 리스트
importances = list(rf_model.feature_importances_)

# (변수, 중요도) 튜플로 구성된 리스트
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(rf_features, importances)]

# 가장 높은 중요도를 가지는 피쳐를 기준으로 내림차순 정렬
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# 변수와 중요도 출력
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: cnt_month_1          Importance: 0.71
Variable: cat_id               Importance: 0.05
Variable: item_id_mean_cnt_1   Importance: 0.05
Variable: shop_id/cat_id_mean_cnt_1 Importance: 0.05
Variable: sales_month          Importance: 0.03
Variable: cnt_month_2          Importance: 0.03
Variable: sub_cat              Importance: 0.02
Variable: month_id             Importance: 0.01
Variable: shop_id              Importance: 0.01
Variable: item_id              Importance: 0.01
Variable: cnt_month_3          Importance: 0.01
Variable: sub_cat_mean_cnt_1   Importance: 0.01
Variable: item_id/city_id_mean_cnt_1 Importance: 0.01
Variable: shop_id/cat_id_mean_cnt_2 Importance: 0.01
Variable: city_id              Importance: 0.0
Variable: main_cat             Importance: 0.0
Variable: year                 Importance: 0.0
Variable: item_id_mean_cnt_2   Importance: 0.0
Variable: item_id_mean_cnt_3   Importance: 0.0
Variable: sub_cat_mean_cnt_2   Importance: 0.0
Variable: sub_cat_mean_cnt_3  

In [16]:
print('Train rmse: ',np.sqrt(mean_squared_error(y_train.clip(0,20), rf_train_pred)))
print('Validation rmse: ',np.sqrt(mean_squared_error(y_valid.clip(0,20), rf_valid_pred)))

Train rmse:  0.8885487032535132
Validation rmse:  0.8321029438914344


In [17]:
print('Test rmse: ',0.95336)

Test rmse:  0.95336
